In [193]:
import requests
import pandas as pd

Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.
a. Use the .find method to find the tag containing the first job title ("Senior Python Developer").

In [194]:
URL = 'https://realpython.github.io/fake-jobs/'
response = requests.get(URL)

In [195]:
requests.get('https://realpython.github.io/fake-jobs/')

<Response [200]>

In [196]:
from bs4 import BeautifulSoup as BS

In [197]:
soup = BS(response.text)

In [198]:
soup.findAll('h2')[0].text

'Senior Python Developer'

b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.

In [199]:
job_titles=soup.findAll('h2')

In [200]:
jobs = [job.get_text() for job in job_titles]

Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", 
a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces 
or other characters at the beginning or end.

In [201]:
job_company=soup.findAll('h3')
jobs_company = [job.get_text() for job in job_company]

job_location=soup.findAll('p')
job_location

In [202]:
job_time=soup.findAll('time')
job_time = [job.get_text() for job in job_time]

d. Take the lists that you have created and combine them into a pandas DataFrame.

In [203]:
job_listings = pd.DataFrame({'Job Title': jobs,'Location': jobs_company,'Date':job_time,'Apply_here':urls})

2 Next, add a column that contains the url for the "Apply" button. Try this in two ways.
a. First, use the BeautifulSoup find_all method to extract the urls.

In [204]:
job_link=soup.findAll('a',href=True)
job_link = [job.get_text() for job in job_link]
len(job_link)

200

urls = []
for link in soup.find_all('a', href=True):
    href = link['href']
    link_text = link.get_text() 
    urls.append(href)
urls

In [205]:
urls = []
for link in soup.find_all('a', href=True):
    href = link['href']
    link_text = link.get_text() 
    if "Apply" in link_text and href.startswith("http"):
        urls.append(href)
len(urls)

100

In [206]:
job_listings = pd.DataFrame({'Job Title': jobs,'Location': jobs_company,'Date':job_time,'Apply_here':urls})

b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. 
Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted 
using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, 
look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [207]:
new_urls = []
base_url='https://realpython.github.io/fake-jobs/jobs/'
for job_id in range(100):
    for job_title in jobs:
        job_titles = job_title.lower().replace(' ', '-')
        job_url = (base_url)+(job_titles)+('-0')+('.html')
        new_urls.append(job_url)

for url in new_urls:
    len(url)

base_urls = []
base_url='https://realpython.github.io/fake-jobs/jobs/'
for job_id in range(100):
    for job_title in jobs:
        job_titles = job_title.lower().replace(' ', '-')
        job_url = f"{base_url}{job_titles}-{job_id}.html"
        base_urls.append(job_url)

for url in base_urls:
    print(url)


3.Finally, we want to get the job description text for each job.
a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.

In [208]:
URL = 'https://realpython.github.io/fake-jobs/jobs/'
response = requests.get(URL)
job_url = 'https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'
response = requests.get(job_url)
soup = BS(response.text, 'html.parser')
job_desc = soup.findAll('p')[1].get_text()
print(job_desc)

Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.


b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".

In [209]:
def job_desc(url):
    response = requests.get(url)
    soup = BS(response.text)
    job_description = soup.find_all('p')[1].get_text()
    return job_description  
    
url = 'https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html'
job_desc(url)

'At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.'

c. Use the .apply method on the url column you created above to retrieve the description text for all of the jobs.

In [210]:
jobs_df = pd.DataFrame(job_listings)

In [211]:
jobs_df.head(2)

,Job Title,Location,Date,Apply_here
0,Senior Python Developer,"Payne, Roberts and Davis",2021-04-08,https://realpython.github.io/fake-jobs/jobs/se...
1,Energy engineer,Vasquez-Davidson,2021-04-08,https://realpython.github.io/fake-jobs/jobs/en...


In [212]:
job_listings['Job_description'] = job_listings['Apply_here'].apply(job_desc)

In [213]:
job_listings.head()

,Job Title,Location,Date,Apply_here,Job_description
0,Senior Python Developer,"Payne, Roberts and Davis",2021-04-08,https://realpython.github.io/fake-jobs/jobs/se...,Professional asset web application environment...
1,Energy engineer,Vasquez-Davidson,2021-04-08,https://realpython.github.io/fake-jobs/jobs/en...,Party prevent live. Quickly candidate change a...
2,Legal executive,"Jackson, Chambers and Levy",2021-04-08,https://realpython.github.io/fake-jobs/jobs/le...,Administration even relate head color. Staff b...
3,Fitness centre manager,Savage-Bradley,2021-04-08,https://realpython.github.io/fake-jobs/jobs/fi...,Tv program actually race tonight themselves tr...
4,Product manager,Ramirez Inc,2021-04-08,https://realpython.github.io/fake-jobs/jobs/pr...,Traditional page a although for study anyone. ...


In [214]:
job_listings.to_csv('job_listings.csv', index=False)